# Web Scrapping with API

In [28]:
import config
import requests

In [29]:
# Create a list to store comment dictionaries
comments_list = []

In [30]:
company = "Company 1"

# Access Database

In [31]:
import pymysql

In [32]:

def connect():
    con=pymysql.connect(host='localhost',user='root',password='',db='companyinfo',charset='utf8mb4')
    return con

In [33]:
con = connect()
stm = con.cursor()

In [34]:
q = f"select * from companydetails where CompanyName='{company}'"
stm.execute(q)

1

In [35]:
rows = stm.fetchall()
rows

(('Company 1',
  'companytest889@gmail.com',
  '61553129864051',
  '122106163322104328',
  '17991588476185435',
  10.0),)

In [36]:
Reciver_Mail = rows[0][1]
page_id = rows[0][2]
FB_post_id = rows[0][3]
INS_post_id = rows[0][4]
prev = rows[0][5]

## (a) Facebook (Meta Graph API)

In [37]:
#Scrapping Facebook Comments
user_access_token = config.FB_user_access_token

url = f'https://graph.facebook.com/v18.0/{page_id}_{FB_post_id}/comments'

params = {
    'access_token': user_access_token,
    'fields': 'message,created_time',
}

response = requests.get(url, params=params)
data = response.json()

# Populate the list with comments
if 'data' in data:
    for comment in data['data']:
        from_info = comment.get('from', {})
        comments_list.append({
                                    'Message': comment['message'],
                                    'Timestamp': comment['created_time'],
                                    'Source' : 'Facebook',
                              })
for comment in comments_list:
    print(comment);

{'Message': 'positive', 'Timestamp': '2023-11-12T07:58:48+0000', 'Source': 'Facebook'}
{'Message': 'helo', 'Timestamp': '2023-11-12T06:44:30+0000', 'Source': 'Facebook'}
{'Message': 'ok', 'Timestamp': '2023-11-12T06:56:52+0000', 'Source': 'Facebook'}
{'Message': 'hey', 'Timestamp': '2023-11-12T07:06:08+0000', 'Source': 'Facebook'}


## (b) Instagram (Meta Graph API)

In [38]:
#Scrapping Instagram comments
user_access_token = config.INS_user_access_token
url = f'https://graph.facebook.com/v18.0/{INS_post_id}/comments'
params = {
    'fields': 'text,timestamp',
    'access_token': user_access_token,
}

response = requests.get(url, params=params)
data = response.json()

# Populate the list with comments
if 'data' in data:
    for comment in data['data']:
        from_info = comment.get('from', {})
        comments_list.append({
                                    'Message':comment['text'],
                                    'Timestamp': comment['timestamp'],
                                    'Source' : 'Instagram',
                            })

for comment in comments_list:
    print(comment);

{'Message': 'positive', 'Timestamp': '2023-11-12T07:58:48+0000', 'Source': 'Facebook'}
{'Message': 'helo', 'Timestamp': '2023-11-12T06:44:30+0000', 'Source': 'Facebook'}
{'Message': 'ok', 'Timestamp': '2023-11-12T06:56:52+0000', 'Source': 'Facebook'}
{'Message': 'hey', 'Timestamp': '2023-11-12T07:06:08+0000', 'Source': 'Facebook'}
{'Message': 'joy mohunbagan', 'Timestamp': '2023-11-13T14:10:08+0000', 'Source': 'Instagram'}
{'Message': 'pic is nice', 'Timestamp': '2023-11-13T14:09:01+0000', 'Source': 'Instagram'}
{'Message': 'it is not good', 'Timestamp': '2023-11-13T13:54:07+0000', 'Source': 'Instagram'}
{'Message': 'its is worst', 'Timestamp': '2023-11-13T13:53:24+0000', 'Source': 'Instagram'}
{'Message': 'hi', 'Timestamp': '2023-11-12T10:53:33+0000', 'Source': 'Instagram'}
{'Message': 'Hemlo', 'Timestamp': '2023-11-12T09:52:54+0000', 'Source': 'Instagram'}


## Sentiment Analysis

## Roberta Pretrained Model

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
from tqdm.notebook import tqdm

In [41]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [42]:
df = pd.DataFrame(comments_list, columns=['Message','Timestamp','Source'])

In [43]:
print(df.dtypes)
df

Message      object
Timestamp    object
Source       object
dtype: object


,Message,Timestamp,Source
0,positive,2023-11-12T07:58:48+0000,Facebook
1,helo,2023-11-12T06:44:30+0000,Facebook
2,ok,2023-11-12T06:56:52+0000,Facebook
3,hey,2023-11-12T07:06:08+0000,Facebook
4,joy mohunbagan,2023-11-13T14:10:08+0000,Instagram
5,pic is nice,2023-11-13T14:09:01+0000,Instagram
6,it is not good,2023-11-13T13:54:07+0000,Instagram
7,its is worst,2023-11-13T13:53:24+0000,Instagram
8,hi,2023-11-12T10:53:33+0000,Instagram
9,Hemlo,2023-11-12T09:52:54+0000,Instagram


In [44]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [45]:
print(df.dtypes)

Message                   object
Timestamp    datetime64[ns, UTC]
Source                    object
dtype: object


In [46]:
df.sort_values(by='Timestamp', inplace = True)

In [47]:
df

,Message,Timestamp,Source
1,helo,2023-11-12 06:44:30+00:00,Facebook
2,ok,2023-11-12 06:56:52+00:00,Facebook
3,hey,2023-11-12 07:06:08+00:00,Facebook
0,positive,2023-11-12 07:58:48+00:00,Facebook
9,Hemlo,2023-11-12 09:52:54+00:00,Instagram
8,hi,2023-11-12 10:53:33+00:00,Instagram
7,its is worst,2023-11-13 13:53:24+00:00,Instagram
6,it is not good,2023-11-13 13:54:07+00:00,Instagram
5,pic is nice,2023-11-13 14:09:01+00:00,Instagram
4,joy mohunbagan,2023-11-13 14:10:08+00:00,Instagram


In [48]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [49]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [50]:
base : float
base = 10.00

res = {
    'pos' : [],
    'neg' : []
}
for i,row in tqdm(df.iterrows(), total=len(df)):
    try:
        print(row);
        text = row['Message']
        #myid = row['Id']
        roberta_result = polarity_scores_roberta(text)
        print(roberta_result)
        if roberta_result['roberta_neg']>roberta_result['roberta_neu'] and roberta_result['roberta_neg']>roberta_result['roberta_pos'] :
            base-=roberta_result['roberta_neg']
            res['neg'].append({
                'Comment' : row['Message'],
                'Source' :row['Source']
            })
        if roberta_result['roberta_pos']>roberta_result['roberta_neu'] and roberta_result['roberta_pos']>roberta_result['roberta_neg'] :
            base+=roberta_result['roberta_pos']
            res['pos'].append({
                'Comment' : row['Message'],
                'Source' :row['Source']
            })
        #print(base);
    except RuntimeError:
        print(f'Broke for id {i}')

  0%|          | 0/10 [00:00<?, ?it/s]

Message                           helo
Timestamp    2023-11-12 06:44:30+00:00
Source                        Facebook
Name: 1, dtype: object
{'roberta_neg': 0.21808101, 'roberta_neu': 0.5567049, 'roberta_pos': 0.2252141}
Message                             ok
Timestamp    2023-11-12 06:56:52+00:00
Source                        Facebook
Name: 2, dtype: object
{'roberta_neg': 0.25796354, 'roberta_neu': 0.49621224, 'roberta_pos': 0.24582422}
Message                            hey
Timestamp    2023-11-12 07:06:08+00:00
Source                        Facebook
Name: 3, dtype: object
{'roberta_neg': 0.21231419, 'roberta_neu': 0.53059936, 'roberta_pos': 0.2570865}
Message                       positive
Timestamp    2023-11-12 07:58:48+00:00
Source                        Facebook
Name: 0, dtype: object
{'roberta_neg': 0.023347158, 'roberta_neu': 0.31671837, 'roberta_pos': 0.65993446}
Message                          Hemlo
Timestamp    2023-11-12 09:52:54+00:00
Source                       Instagr

In [66]:
print(base)

9.667113244533539


# Update Database

In [70]:
q = f"update companydetails set Score = {base} where CompanyName = '{company}'"
stm.execute(q)
con.commit()

# Sending Email

In [55]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [56]:
def send_email(subject, body, to_email, smtp_server, smtp_port, smtp_username, smtp_password):
    # Create the MIME object
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)

    server.sendmail(smtp_username, to_email, msg.as_string())
    server.quit()

In [57]:
subject = "Change In Score Detected"
body = f"New Score : {base} \nDelta change : {base-prev} \nLast 5 negitive comments:{res['neg'][-5:]} \nLast 5 positive comments:{res['pos'][-5:]}"

In [58]:
smtp_server = "smtp.gmail.com"
smtp_port = 587 

In [59]:
if(prev!=base) :
    send_email(subject=subject,
            body=body,
            to_email=Reciver_Mail,
            smtp_server=smtp_server,
            smtp_port=smtp_port,
            smtp_username=config.Sender_mail,
            smtp_password=config.AppPassword)
    print("Mail send")

In [71]:
stm.close()
con.close()